In [89]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [90]:
df = pd.read_csv('short.csv')

In [91]:
df['action'].value_counts()

action
find_doctor         4000
find_medicalshop    4000
find_hospital       4000
find_lab            4000
book_appointment    4000
my_appointments     4000
my_records          4000
Name: count, dtype: int64

In [92]:
label_encoder = LabelEncoder()
df['action'] = label_encoder.fit_transform(df['action'])

In [93]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [94]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['query'])
total_words = len(tokenizer.word_index) + 1

In [95]:
train_sequences = tokenizer.texts_to_sequences(train_data['query'])
test_sequences = tokenizer.texts_to_sequences(test_data['query'])

In [96]:
train_padded_sequences = pad_sequences(train_sequences)
test_padded_sequences = pad_sequences(test_sequences, maxlen=train_padded_sequences.shape[1])

In [97]:
df['action'].unique()

array([1, 4, 2, 3, 0, 5, 6])

In [98]:
train_labels = tf.keras.utils.to_categorical(train_data['action'], num_classes=len(df['action'].unique()))
test_labels = tf.keras.utils.to_categorical(test_data['action'], num_classes=len(df['action'].unique()))

In [99]:
model = Sequential()
model.add(Embedding(total_words, 64, input_length=train_padded_sequences.shape[1]))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(len(set(df['action'])), activation='softmax'))

In [100]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [109]:
model.fit(train_padded_sequences, train_labels, epochs=2, verbose=1, validation_data=(test_padded_sequences, test_labels))

Epoch 1/2
700/700 [==============================] - 3s 4ms/step - loss: 0.7410 - accuracy: 0.6640 - val_loss: 0.7642 - val_accuracy: 0.6464
Epoch 2/2
700/700 [==============================] - 3s 4ms/step - loss: 0.7381 - accuracy: 0.6666 - val_loss: 0.7572 - val_accuracy: 0.6561


In [146]:
new_queries = ["find a hospital in ranchi for tommorow 10 pm", "i have to refer my medical records please find some"]

In [147]:
new_sequences = tokenizer.texts_to_sequences(new_queries)
new_padded_sequences = pad_sequences(new_sequences, maxlen=train_padded_sequences.shape[1])

In [148]:
predictions = model.predict(new_padded_sequences)
predicted_classes = [label_encoder.classes_[tf.argmax(prediction).numpy()] for prediction in predictions]

1/1 [==============================] - 0s 18ms/step


In [149]:
predicted_classes

['find_hospital', 'my_records']

In [106]:
model.evaluate(test_padded_sequences, test_labels)

175/175 [==============================] - 0s 2ms/step - loss: 0.7790 - accuracy: 0.6480


[0.7790299654006958, 0.6480357050895691]

In [107]:
model.save('action.keras')